In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm 

N_SAMPLES = 10000  # Number of samples per simulation run
N_SIMULATIONS = 1000  # Number of simulation runs to average results

# True DGP parameters
TRUE_BETA_0 = 1.0
TRUE_BETA_S = 0.5
TRUE_BETA_Z = 1.0
TRUE_BETA_X = 0.7
TRUE_DELTA_S = 0.8  # Z = delta_S * S + epsilon_Z
TRUE_GAMMA_S = 0.6  # X = gamma_S * S + epsilon_X

# Variances of error terms and S (using standard deviations here for clarity in generation)
SIGMA_S = 1.0  # Standard deviation for S (S ~ N(0, 1^2))
SIGMA_Z_EPS = 0.1  # Standard deviation for epsilon_Z
SIGMA_X_EPS = 0.1  # Standard deviation for epsilon_X
SIGMA_Y_EPS = 0.1  # Standard deviation for epsilon_Y

# --- Helper functions ---

def calculate_demographic_parity(y_pred, s_binary):
    """
    Calculates Demographic Parity (DP) for continuous predictions or labels.
    Assumes s_binary is already binarized (e.g., 0 and 1).
    """
    s_unique = np.unique(s_binary)
    if len(s_unique) < 2:
        return 0.0  # Cannot calculate DP with less than 2 sensitive groups

    # Get mean prediction for each sensitive group
    mean_preds_group0 = np.mean(y_pred[s_binary == s_unique[0]])
    mean_preds_group1 = np.mean(y_pred[s_binary == s_unique[1]])
    
    return np.abs(mean_preds_group1 - mean_preds_group0)


def _compute_conditional_demographic_parity(y_pred_probs, s_binary, z, bin_size=20, min_group_samples=50):
    """
    Computes Conditional Demographic Parity (CDP) based on admissible features Z.
    This version is robust to sparse Z-bins by enforcing a minimum number of samples.
    """
    # For a linear model, y_pred_probs are directly the predicted outcomes.
    y_pred = y_pred_probs  # Use raw predicted outcomes for mean calculations

    # Ensure s_binary and z are 1D arrays for consistency if they are single features
    if s_binary.ndim > 1 and s_binary.shape[1] == 1: s_binary = s_binary.flatten()
    if z.ndim > 1 and z.shape[1] == 1: z = z.flatten()

    s_unique = np.unique(s_binary)
    if len(s_unique) < 2:
        return 0.0  # Cannot compute CDP if there's only one sensitive group after binarization

    df = pd.DataFrame({
        'y_pred': y_pred,
        's': s_binary  # Use the binarized S
    })
    
    z_df_binned = pd.DataFrame(index=df.index)  # Create an empty DataFrame for binned Z features
    
    # Bin continuous features in Z to create discrete groups for conditioning
    # Iterate over columns of the original z array if it's 2D, or just z if 1D
    if z.ndim > 1:
        for i, col_data in enumerate(z.T):  # Iterate over columns if z is 2D
            col_name = f'z_col_{i}'
            if pd.api.types.is_numeric_dtype(col_data) and len(np.unique(col_data)) > bin_size:
                try:
                    z_df_binned[col_name] = pd.qcut(col_data, bin_size, labels=False, duplicates='drop')
                except ValueError:
                    z_df_binned[col_name] = pd.cut(col_data, bin_size, labels=False, duplicates='drop')
            else:
                z_df_binned[col_name] = col_data
    else:  # z is 1D
        col_name = 'z_col_0'
        if pd.api.types.is_numeric_dtype(z) and len(np.unique(z)) > bin_size:
             try:
                z_df_binned[col_name] = pd.qcut(z, bin_size, labels=False, duplicates='drop')
             except ValueError:
                z_df_binned[col_name] = pd.cut(z, bin_size, labels=False, duplicates='drop')
        else:
            z_df_binned[col_name] = z
    
    df = pd.concat([df, z_df_binned], axis=1)

    max_cdp = 0.0
    
    # Group by the conditional features (Z)
    # Iterate through each unique combination of binned conditional features (Z)
    # Filter out combinations where any Z-column is NaN (due to duplicates='drop' on qcut/cut)
    # This ensures we only group on valid Z bins.
    valid_z_groups = df.dropna(subset=list(z_df_binned.columns))

    for z_group_key, group_df in valid_z_groups.groupby(list(z_df_binned.columns)):
        
        # Check if all sensitive groups are present AND meet minimum sample size within this Z-group
        all_s_present_and_sufficient = True
        rates_for_s_groups = {}
        
        for s_val in s_unique:
            s_sub_group = group_df[group_df['s'] == s_val]
            if len(s_sub_group) < min_group_samples:  # Check minimum samples
                all_s_present_and_sufficient = False
                break
            rates_for_s_groups[s_val] = s_sub_group['y_pred'].mean()
        
        if all_s_present_and_sufficient:
            # Calculate all pairwise differences within this Z-group
            for i in range(len(s_unique)):
                for j in range(i + 1, len(s_unique)):
                    s1 = s_unique[i]
                    s2 = s_unique[j]
                    
                    rate1 = rates_for_s_groups[s1]
                    rate2 = rates_for_s_groups[s2]
                    
                    max_cdp = max(max_cdp, abs(rate1 - rate2))
    
    return max_cdp


# --- Data Generation Function ---
def generate_data(n_samples):
    # Generate S from a Normal distribution
    s = np.random.normal(0, SIGMA_S, n_samples)
    
    # Generate Z and X based on S and independent errors
    epsilon_z = np.random.normal(0, SIGMA_Z_EPS, n_samples)
    z = TRUE_DELTA_S * s + epsilon_z
    
    epsilon_x = np.random.normal(0, SIGMA_X_EPS, n_samples)
    x = TRUE_GAMMA_S * s + epsilon_x
    
    # Generate Y based on S, Z, X and independent error
    epsilon_y = np.random.normal(0, SIGMA_Y_EPS, n_samples)
    y = TRUE_BETA_0 + TRUE_BETA_S * s + TRUE_BETA_Z * z + TRUE_BETA_X * x + epsilon_y
    
    return s, z, x, y

# --- Simulation Loop ---
simulation_results = []

# Calculate necessary variances and covariances based on DGP parameters for analytical expectations
# These are population variances/covariances based on the DGP equations
V_S = SIGMA_S**2
V_Z_analytical = TRUE_DELTA_S**2 * V_S + SIGMA_Z_EPS**2
V_X_analytical = TRUE_GAMMA_S**2 * V_S + SIGMA_X_EPS**2
Cov_ZS_analytical = TRUE_DELTA_S * V_S
Cov_XS_analytical = TRUE_GAMMA_S * V_S
Cov_ZX_analytical = TRUE_DELTA_S * TRUE_GAMMA_S * V_S

# Denominator for Naive OVB (analytical)
Delta_naive_analytical = V_Z_analytical * V_X_analytical - Cov_ZX_analytical**2

for _ in tqdm(range(N_SIMULATIONS), desc="Running Linear Simulations"):
    s, z, x, y = generate_data(N_SAMPLES)
    
    # Add intercept for statsmodels OLS (for regressors that might be used alone or in combinations)
    X_s_only = sm.add_constant(s, prepend=True, has_constant='add')
    
    # Create a dummy (binary) sensitive attribute for DP/CDP calculation,
    # as these metrics are typically defined for binary groups.
    # We'll use the median of S to binarize.
    s_binary = (s > np.median(s)).astype(int)

    # --- 1. Naive Model (Y ~ Z + X) ---
    X_naive_model = sm.add_constant(np.column_stack((z, x)), prepend=True, has_constant='add')
    
    model_naive = sm.OLS(y, X_naive_model).fit()
    y_pred_naive = model_naive.predict(X_naive_model)

    naive_beta_Z = model_naive.params[1]  # Beta for Z
    naive_beta_X = model_naive.params[2]  # Beta for X

    dp_naive = calculate_demographic_parity(y_pred_naive, s_binary)
    cdp_naive = _compute_conditional_demographic_parity(y_pred_naive, s_binary, z)

    ssr_naive = np.sum((y - y_pred_naive)**2)
    tss = np.sum((y - np.mean(y))**2)
    r2_naive = 1 - ssr_naive / tss

    # --- 2. Full Debias (Y ~ Z_res + X_res) ---
    model_z_on_s = sm.OLS(z, X_s_only).fit()
    z_res = z - model_z_on_s.predict(X_s_only)

    model_x_on_s = sm.OLS(x, X_s_only).fit()
    x_res = x - model_x_on_s.predict(X_s_only)
    
    X_full_debias = sm.add_constant(np.column_stack((z_res, x_res)), prepend=True, has_constant='add')
    model_full_debias = sm.OLS(y, X_full_debias).fit()
    y_pred_full_debias = model_full_debias.predict(X_full_debias)

    full_debias_beta_Z_res = model_full_debias.params[1]  # Beta for Z_res
    full_debias_beta_X_res = model_full_debias.params[2]  # Beta for X_res

    dp_full_debias = calculate_demographic_parity(y_pred_full_debias, s_binary)
    cdp_full_debias = _compute_conditional_demographic_parity(y_pred_full_debias, s_binary, z)

    ssr_full = np.sum((y - y_pred_full_debias)**2)
    r2_full = 1 - ssr_full / tss

    # --- 3. Our Method (Fit Y ~ Z + X_res + S, predict with S=0) ---
    # Residualize only X (inadmissible)
    # Fit with S to control for direct/inadmissible effects
    X_ours = sm.add_constant(np.column_stack((z, x_res, s)), prepend=True, has_constant='add')
    model_ours = sm.OLS(y, X_ours).fit()

    ours_beta_Z = model_ours.params[1]  # Beta for Z
    ours_beta_X_res = model_ours.params[2]  # Beta for X_res
    # beta_S = model_ours.params[3]  # Not reported, but = beta_S + beta_X * gamma_S

    # Predict by setting S to 0 (average over marginal S)
    X_pred = sm.add_constant(np.column_stack((z, x_res, np.zeros_like(s))), prepend=True, has_constant='add')
    y_pred_ours = model_ours.predict(X_pred)

    dp_ours = calculate_demographic_parity(y_pred_ours, s_binary)
    cdp_ours = _compute_conditional_demographic_parity(y_pred_ours, s_binary, z)

    ssr_ours = np.sum((y - y_pred_ours)**2)
    r2_ours = 1 - ssr_ours / tss

    # Store results for this simulation run
    simulation_results.append({
        'naive_R2': r2_naive,
        'naive_beta_Z': naive_beta_Z,
        'naive_beta_X': naive_beta_X,
        'naive_DP': dp_naive,
        'naive_CDP': cdp_naive,

        'full_debias_R2': r2_full,
        'full_debias_beta_Z_res': full_debias_beta_Z_res,
        'full_debias_beta_X_res': full_debias_beta_X_res,
        'full_debias_DP': dp_full_debias,
        'full_debias_CDP': cdp_full_debias,

        'ours_R2': r2_ours,
        'ours_beta_Z': ours_beta_Z,
        'ours_beta_X_res': ours_beta_X_res,
        'ours_DP': dp_ours,
        'ours_CDP': cdp_ours,
    })

results_df = pd.DataFrame(simulation_results)

summary_mean = results_df.mean()
summary_std = results_df.std()

print("\n--- Simulation Results Summary (Mean ± Std Dev) ---")
summary_table = pd.DataFrame({'mean': summary_mean, 'std': summary_std})
print(summary_table)

Running Linear Simulations: 100%|██████████| 1000/1000 [00:50<00:00, 19.90it/s]


--- Simulation Results Summary (Mean ± Std Dev) ---
                            mean       std
naive_R2                0.995816  0.000083
naive_beta_Z            1.396027  0.006894
naive_beta_X            0.997019  0.009128
naive_DP                2.735736  0.021247
naive_CDP               0.153767  0.011365
full_debias_R2          0.004996  0.000129
full_debias_beta_Z_res  0.999849  0.009830
full_debias_beta_X_res  0.699731  0.009976
full_debias_DP          0.001198  0.000877
full_debias_CDP         0.163022  0.010139
ours_R2                 0.712808  0.004889
ours_beta_Z             0.999849  0.009830
ours_beta_X_res         0.699731  0.009976
ours_DP                 1.275911  0.015924
ours_CDP                0.017950  0.006248
